# Extracting prescription quantity and frequency.
The purpose of this notebook is to develop a script to extract presctiption quantity and frequency from the free-test field `medicationdosage` in the `tbl_srprimarycaremedication` table.

Some pharmacy lingo will be needed. The initialisms below might be spearated by periods, `.`, or slashes, `/`, and are sometimes capitalised. They can also be concatonated without spaces, e.g. achs = ac + hs = before meals and at bedtime.
| Initialism | Meaning |
| ------- | ------- |
| a | ante, before |
| ac | ante cibum (before food) |
| ad lib | as desired |
| am | in the morning |
| aq | aqua, take with water |
| bib | drink |
| bd, bds, bid | bis die (twice daily) |
| et | and |
| c\* | with \* (where \* stands for the other thing) |
| cc | cum cibos (with food) |
| cap | capsule |
| h | hour |
| hs | bedtime or half-strength |
| i, ii, iii, ... | 1, 2, 3, etc. tablets |
| id | intradermally, through the skin |
| im | intramuscular, inject into muscle |
| in | intranasal, through the nose |
| inh | inhaler |
| inj | injection |
| neb | nebuliser, inhalant medication |
| nbm, npo | nill by mouth |
| nocte, noct | at night |
| od | omni die (every day) |
| om | omni mane (every morning) |
| on | omni nocte (every night) |
| p | post, after |
| pc | post cibum (after food) |
| pm | in the evening |
| pn | per neb, by nebulizer |
| po | per os, orally, by mouth |
| pr | rectally 
| prn | pro re nata (when required) |
| q\* | every \* (where \* is a wildcard for a timepoints or  intervals, e.g. q2h = every 2 hours, q1d = qd = every day, qam = every morning) |
| qds, qid | quater die sumendum (to be taken four times daily) |
| qod | every other day |
| qs | a sufficient amount |
| qqh | quarta quaque hora (every four hours) |
| ss | half |
| s\* | sans\*, without\* (where \* stands for the other thing) |
| sc | subcutaneous, inject under the skin |
| stat | immediately |
| syr | syrup |
| tab | tablet |
| tds, tid | ter die sumendum (to be taken three times daily) |
| u | unit |
| ud | as directed |
| w\* | with \* (where \* stands for the other thing) |
| wf | with food |
| x\* | if \* is a number, then \*-many tablets. If \* is a number and/or letter, then \*-many tablets per interval (e.g. d = day) or \*-many intervals |


Sources:

- https://bnf.nice.org.uk/about/abbreviations-and-symbols/
- https://www.nhs.uk/nhs-app/nhs-app-help-and-support/health-records-in-the-nhs-app/abbreviations-commonly-found-in-medical-records/
- https://www.drugs.com/article/prescription-abbreviations.html#
- https://chartercollege.edu/news-hub/72-abbreviations-every-pharmacy-tech-needs-know/
- https://mn.gov/boards/assets/prescription%20abbreviations_tcm21-26882.pdf


## Strategy

### Morning, evening, before bed, etc.
I want a total count of tablets per day so I need to combine the morning, evening, etc. prescriptions. My plan is to parse these prescriptions into separate columns that I will later sum:

| n_morning | n_evening | n_night | n_bedtime | n_mealtime\* |
| --------- | --------- | ------- | --------- | ------------ |
| ...       | ...       | ...     | ...       | ...          |

\*`n_ MEALTIME ` will include "before food", "after food", and "with food".


### Units vs tablets.
I will treat a tablet as a unit so that there is a common language.



Install packages.

In [1]:
#######################
## Install packages. ##
#######################
library( pacman )
pacman::p_load( bigrquery, tidyverse, readr )
bq_auth()

Set and load requisites.

In [2]:
##############################
## Set and load requisites. ##
##############################

# Setup connection to GCP.
project_id <- "yhcr-prd-phm-bia-core"
con <- DBI::dbConnect( drv = bigquery(), project = project_id )

# Define R tibbles from GCP tables.
r_tbl_BNF_DMD_SNOMED_lkp <- dplyr::tbl( con, "CB_LOOKUPS.tbl_BNF_DMD_SNOMED_lkp" )
r_tbl_srprimarycaremedication <- dplyr::tbl( con, "CB_FDM_PrimaryCare_V7.tbl_srprimarycaremedication" )

# Clinical code lists (BNF, SNOMED-CT, etc).
codes_BNF_meds_of_interest <-
    readr::read_csv(file = 'ciaranmci-bnf-section-61-drugs-for-diabetes-207573b7.csv',
                    col_types = cols( code = col_character(), term = col_character() ) )$code
names_meds_of_interest <-
    r_tbl_BNF_DMD_SNOMED_lkp %>%
    dplyr::filter( BNF_Code %in% codes_BNF_meds_of_interest ) %>%
    dplyr::select( DMplusD_ProductDescription )

Warning message:
“<BigQueryConnection> uses an old dbplyr interface
ℹ Please install a newer version of the package or contact the maintainer
This warning is displayed once every 8 hours.”


Apply transforms to the free-text in the `medicationdosage` column to get the contents into a machine-readable format. Also, work with the `medicationquantity` columns to see if you can determine the expected prescription duration.

In [77]:
n_row_display <- 200
options(repr.matrix.max.rows = n_row_display)

qry <-
r_tbl_srprimarycaremedication %>%
# Select every record that has a prescription for any diabetes medication.
dplyr::inner_join( names_meds_of_interest, by = join_by( nameofmedication == DMplusD_ProductDescription ) ) %>%
# Filter out all 'as directed' dosages.
dplyr::filter( sql( 'NOT REGEXP_CONTAINS(medicationdosage, r\'(as directed)|(as dir)|(asd)|(mdu)|(MDU)|(m.d.u.)|(M.D.U.)\' )' ) ) %>%
#dplyr::filter( sql( 'NOT REGEXP_CONTAINS(medicationdosage, r\'((as\\ )?directed)|(asd)|(mdu)|(MDU)|(m.d.u.)|(M.D.U.)|(as dir)\' )' ) ) %>%
# Filter for tablets and capsules, only.
dplyr::filter( medicationquantity %LIKE% '%tablet%' %OR% medicationquantity %LIKE% '%capsule%' ) %>%
# Filter out blank prescriptions.
dplyr::filter( medicationdosage != "" ) %>%
# Filter out prosaic dosages that give week-by-week explanations.
dplyr::filter( sql( 'NOT REGEXP_CONTAINS(medicationdosage, r\'(on\\ )?w(e*)?ks?\\ ?(-)?[0-9]?\' )' ) ) %>%
# Select unique `medicationdosage` values, only.
dplyr::select( medicationdosage, medicationquantity ) %>%
dplyr::distinct() %>%

dplyr::mutate(rn = row_number()) %>%

#####################
## Transformations ##
#####################

# Work with the `medicationdosage` columns to see if you can determine the expected prescription duration.
### Transfrom set 1. ###
dplyr::mutate(
    dosage_transformed = 
    medicationdosage %>%
    
    # Transform X: Remove periods, `.`.
    ###stringr::str_replace(., "\\.",              "" ) %>%
    stringr::str_replace("\\.",                                                           "" ) %>%

    # Transform X: Make everything lower case.
    stringr::str_to_lower() %>%

    # Transform X: "and " to +.
    stringr::str_replace( "\\ (and)|\\&\\ ",                                              " + " ) %>%
    
    # Transform X: Collapse numbers and their units so that the numbers don't get interpretted as counts, later.
    stringr::str_replace( "(gram)s?",                                                     "g" ) %>%
    stringr::str_replace( "[0-9]*\\ ?(mg|g)\\ |$",                                        "" ) %>%
    
    # Transform X: Remove any instruction about X-many minutes before or after a meal.
    stringr::str_replace( "[0-9]*\\ ?(min)(ute)?s?",                                      "" ) %>%
    stringr::str_replace( "\\ |^[0-9]*\\ ?h(ou)?rs?\\ |$",                                "" ) %>%
    
    # Transform X: Convert time-of-day instructions to latin capitalised initialisms, or latin words.
    stringr::str_replace( "(pre-)",                                                       "") %>%#"before" ) %>%
    stringr::str_replace( "(in the )?(morning)",                                          "") %>%#" WHEN = (A.M.) " ) %>%
    stringr::str_replace( "\\ (morn)\\ ",                                                 "") %>%#" WHEN = (A.M.) " ) %>%
    stringr::str_replace( "(at )?(morning)[a-z]?",                                        "") %>%#" WHEN = (A.M.) " ) %>%
    stringr::str_replace( "[0-9]\\ (o.m.)",                                               "") %>%#" WHEN = (A.M.) " ) %>%
    stringr::str_replace( "[0-9]\\ (om)",                                                 "") %>%#" WHEN = (A.M.) " ) %>%
    stringr::str_replace( "(\\ |^[0-9])?(am)(\\ |$)",                                     "") %>%#" WHEN = (A.M.) " ) %>%
    stringr::str_replace( "((ma)n*e)",                                                    "") %>%#" WHEN = (A.M.) " ) %>%

    stringr::str_replace( "(in the evening)",                                             "") %>%#" WHEN = (P.M.) " ) %>%
    stringr::str_replace( "(midday)",                                                     "") %>%#" WHEN = (P.M.) " ) %>%
    stringr::str_replace( "(evening)[a-z]?",                                              "") %>%#" WHEN = (P.M.) " ) %>%
    stringr::str_replace( "\\ (eve)\\ |$",                                                "") %>%#" WHEN = (P.M.) " ) %>%
    stringr::str_replace( "\\ ?(pm)$?",                                                   "") %>%#" WHEN = (P.M.) " ) %>%

    stringr::str_replace( "(noct)e?",                                                     "") %>%#" WHEN = (NOCTE) " ) %>%
    stringr::str_replace( "(at )?(in the )?(night)",                                      "") %>%#" WHEN = (NOCTE) " ) %>%
    stringr::str_replace( "(o.n.)",                                                       "") %>%#" WHEN = (NOCTE) " ) %>%
    stringr::str_replace( "\\ (on)\\ |$",                                                 "") %>%#" WHEN = (NOCTE) " ) %>% # Not confident about replacing "on" with NOCTE.
    stringr::str_replace( "(wf)|(cc)|(pc)|(cc&pc)",                                       "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(before)?(after)?(with)?\\ (food)s?",                          "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(before)?(after)?(with)?\\ (me)((al)|(la))s?",                 "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(before)?(after)?(with)?\\ (break)\\ ?(fast)s?",               "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(before)?(after)?(with)?\\ (lunch)(time)?",                    "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(eve)(ning)?\\ (meal)",                                        "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(before)?(after)?(with)?\\ (dinner)s?",                        "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(before)?(after)?(with)?\\ (supper)s?",                        "") %>%#" WHEN = (MEALTIME) " ) %>%
    stringr::str_replace( "(before)?(after)?(with)?\\ ?(tea)\\ ?(time)?s?(\\ meal)?",     "") %>%#" WHEN = (MEALTIME) " ) %>%

    # Transfrom X: Tidy away typos.
    stringr::str_replace( "(ty)[a-z]",                                                    "t" ) %>% # Still experimental. This is trying to fix typos like "tywo" instead of "two".
    stringr::str_replace( "(afte)r?",                                                     "" ) %>%
    stringr::str_replace( "(befor)e?",                                                    "" ) %>%
    stringr::str_replace( "(me)[a-z]?(al)s?",                                             "" ) %>%
    stringr::str_replace( "\\ (ime)",                                                     "" ) %>%
    stringr::str_replace( "\\ (-time)",                                                   "" ) %>%
    stringr::str_replace( "\\ (time)\\ ",                                                 "" ) %>%
    stringr::str_replace( "(eve)[a-z](ing)",                                              "" ) %>%
    stringr::str_replace( "(m[a-z]?o(nr|rn)?i?ng)",                                       "" ) %>%

    # Transform X: Handle special case of "for one week".
    stringr::str_replace( "(for)?\\ (one)?1?\\ (week)s?",                                 " INTERVAL = (1) " ) %>% # Ideally, this set of transforms would handle all the "week1", "week 2", etc. instructions.
    stringr::str_replace( "(for)?\\ (two)?2?\\ (week)s?",                                 " INTERVAL = (2) " ) %>% 
    stringr::str_replace( "(for)?\\ (three)?3?\\ (week)s?",                               " INTERVAL = (3) " ) %>% 
    stringr::str_replace( "(for)?\\ (four)?4?\\ (week)s?",                                " INTERVAL = (4) " ) %>% 

    # Transfrom X: Homogenise intervals, e.g. hourly, daily.
    stringr::str_replace( "(a day)",                                                      "daily" ) %>%
    stringr::str_replace( "(aday)",                                                       "daily" ) %>%
    stringr::str_replace( "(each day)",                                                   "daily" ) %>%
    stringr::str_replace( "(per day)",                                                    "daily" ) %>%
    stringr::str_replace( "/\\ ?(day)",                                                   "daily" ) %>%
    stringr::str_replace( "(every day)",                                                  "daily" ) %>%
    stringr::str_replace( "(^|\\ )(od)\\ ",                                               "daily" ) %>%
    stringr::str_replace( "[0-9]d$",                                                      "daily" )

) %>%

### Transfrom set 2. ###
dplyr::mutate_at(
    .vars = vars( dosage_transformed )
    ,.funs = funs(
        dosage_transformed %>%
         
        # Transform X: Remove unnecessary 'tablet'.
        stringr::str_replace( "\\ ?(tab)(le[a-z]?t)?s?(\\(s\\))?",                            "" ) %>%


        # Transform X: Remove unnecessary 'take' and its typos.
        stringr::str_replace( "(ta)[a-z]e?(ing)?n?",                                          "" ) %>%

        # Transform X: Remove unnecessary ' to be '.
        stringr::str_replace( "\\ ?(to be)\\ ",                                               "" ) %>%

        # Transform X: Remove unnecessary 'use '.
        stringr::str_replace( "(use)[0-9]?\\ ",                                               "" ) %>%

        # Transform X: Ignore 'working up to' and its variants.
        #stringr::str_replace( "work(ing)? up to ",  "" ) %>% # This might be important information, so I'll leave it in for now.
        #stringr::str_replace( "wean up gently to",  "" ) %>%
        #stringr::str_replace( "work up slowly to",  "" ) %>%
        # (up)\\ ?(titrate)

        # Transform X: Ignore notions of 'with'.
        stringr::str_replace( "(with)/?\\ ?(the)?",                                           "" ) %>%
        stringr::str_replace( "(w/)\\ ",                                                      "" ) %>%
        stringr::str_replace( "\\ c\\ ",                                                      "" ) %>%
        stringr::str_replace( "\\ ?w\\ ",                                                     "" ) %>%

        # Transform X: Ignore 'as needed' and similar.
        stringr::str_replace( "(as needed)",                                                  "" ) %>%
        stringr::str_replace( "(when needed)",                                                "" ) %>%
        stringr::str_replace( "(as necessary)",                                               "" ) %>%
        stringr::str_replace( "(when necessary)",                                             "" ) %>%
        stringr::str_replace( "(as required)",                                                "" ) %>%
        stringr::str_replace( "(when required)",                                              "" ) %>%
        stringr::str_replace( "(when req)",                                                   "" ) %>%
        stringr::str_replace( "(as desired)",                                                 "" ) %>%
        stringr::str_replace( "(when desired)",                                               "" ) %>%
        stringr::str_replace( "(ad lib)",                                                     "" ) %>%
        stringr::str_replace( "( qs )",                                                       "" ) %>%

        # Transform X: Fix typos.
        stringr::str_replace( "!",                                                            "1" ) %>%
        # Transform X: Leading quantities.
        stringr::str_replace( "1",                                                            " QUANT = (1) " ) %>%
        stringr::str_replace( "2",                                                            " QUANT = (2) " ) %>%
        stringr::str_replace( "3",                                                            " QUANT = (3) " ) %>%
        stringr::str_replace( "4",                                                            " QUANT = (4) " ) %>%
        stringr::str_replace( "5",                                                            " QUANT = (5) " ) %>%

        # Transform X: Convert multiplier words to digits. I assume a daily base for my frequency, e.g. daily = 1.
    # I've having trouble where these are replacing the numbers within my parentheses for QUANT.
        stringr::str_replace( "\\ ?(times)",                                                  "" ) %>%
        stringr::str_replace( "(once)|(daily)|((\\ x)?(od)s?\\ ?)",                           " FREQ = (1) " ) %>% # The preceding space is not optional because, otherwise, we transform "blood" to "blo FREQ = (1)".
        stringr::str_replace( "(twice)|((\\ x)?(bd)s?\\ ?)",                                  " FREQ = (2) " ) %>%
        stringr::str_replace( "\\ ?x?[0-9]?ti?ds?\\ ?",                                       " FREQ = (3) " ) %>%
        stringr::str_replace( "(every\\ 8\\ hours)",                                          " FREQ = (3) " ) %>%
        stringr::str_replace( "\\ ?x?[0-9]?(qd)s?\\ ?",                                       " FREQ = (4) " ) %>%
    
#        # Transform X: Leading quantities.
#        stringr::str_replace( "^|\\ (one)\\ |$",                                              " QUANT = (1) " ) %>%
#        stringr::str_replace( "^|\\ (two)\\ |$",                                              " QUANT = (2) " ) %>%
#        stringr::str_replace( "^|\\ (three)\\ |$",                                            " QUANT = (3) " ) %>%
#        stringr::str_replace( "^|\\ (four)\\ |$",                                             " QUANT = (4) " ) %>%
#        stringr::str_replace( "^|\\ (five)\\ |$",                                             " QUANT = (5) " ) %>%  
        
        # Transform X: Convert number words to digits.
        stringr::str_replace( "(half)|(every alternate day)|(alt days)|(every other day)", " QUANT = (\u00BD) " ) %>%
        stringr::str_replace( "(one)|(\\ |^)i\\ |(whole)",                                     " QUANT = (1) " ) %>%
        stringr::str_replace( "(two)|(\\ |^)ii\\ ",                                            " QUANT = (2) " ) %>%
        stringr::str_replace( "(three)|(\\ |^)iiii\\ ",                                        " QUANT = (3) " ) %>%
        stringr::str_replace( "(four)|(\\ |^)iv\\ ",                                           " QUANT = (4) " ) %>% # I hope patients don't take more than 4 of the same tablet in a day.
        
        # Transform X: Remove leading 'x' in front of digits.
        stringr::str_replace( "(^|x\\ ?)1",                                                    " QUANT = (1) " ) %>%
        stringr::str_replace( "(^|x\\ ?)2",                                                    " QUANT = (2) " ) %>%
        stringr::str_replace( "(^|x\\ ?)3",                                                    " QUANT = (3) " ) %>%
        stringr::str_replace( "(^|x\\ ?)4",                                                    " QUANT = (4) " ) %>%
        
        # Transform X: Remove schpeel about "your practice will contact you..."
        stringr::str_replace( "(your practice will contact you).*$",                           "" ) %>%
        
        # Transform X: Remove schpeel about "whilst taking steriods...
        stringr::str_replace( "(whilst taking steroids)",                                      "" )
        
        )
    ) %>%

### Transfrom set END. ###
dplyr::mutate_at(
    .vars = vars( dosage_transformed )
    ,.funs = funs(
        dosage_transformed %>%
        # Transfrom X: Remove typos that results from other transforms.
        stringr::str_replace( "\\ (in)\\ |$",                                                  "" ) %>%
        stringr::str_replace( "\\ (in)\\ |$",                                                  "" ) %>% # Need to do this twice because stringr::str_replace_all() isn't possible in SQL.
        stringr::str_replace( "\\ (at)",                                                       "" ) %>%
        stringr::str_replace( "(then)",                                                        "" ) %>%
        stringr::str_replace( "\\ (time)",                                                     "" ) %>%
        stringr::str_replace( "\\ (just)",                                                     "" ) %>%
        stringr::str_replace( "(every)",                                                       "" ) %>%
        stringr::str_replace( "(main)",                                                        "" ) %>%
        stringr::str_replace( "(each)",                                                        "" ) %>%
        stringr::str_replace( "(evenin)",                                                      "" ) %>%
        stringr::str_replace( "(of)\\ (the)?(day)",                                            "" ) %>%
        stringr::str_replace( "(for)\\ (diabetes)",                                            "" ) %>%
        stringr::str_replace( "(day)|(dy)|(da)|(hr)|( d)",                                     "" ) %>%
        stringr::str_replace( " es ",                                                          "" ) %>%
        stringr::str_replace( "(\\(\\))",                                                      "" ) %>%
        stringr::str_replace( "\\ \\+\\ |$",                                                   "" ) %>%
        stringr::str_replace( "\\ \\ ",                                                        " " ) %>%
        stringr::str_replace( "\\ -\\ ",                                                       " " ) %>%
        stringr::str_replace( "^|\\ (a|x)\\ |$",                                               " " )
        )
    ) %>%
        
        
 


### DISPLAY ###
distinct() %>% collect() %>% head( n = n_row_display ) %>% suppressWarnings()
qry %>% select(rn, medicationdosage, dosage_transformed)

##############
### TRICKY ###
##############
# 1-2 IN THE MORNING
# "time" is becoming "WHEN = (A.M.)"
# "in afternoon" becomes "in no"
# "two mane" becaomes "tWHEN = (A.M.) ne"
# "1 TABLET AT 10 AM ONE TABLET AT 4 PM" becomes "QUANT = (1) QUANT = (1) 0 QUANT = (1) QUANT = (4)"

rn,medicationdosage,dosage_transformed
<int>,<chr>,<chr>
3,tds,FREQ = (3)
4,take 2 three times daily,QUANT = (2) QUANT = (3) FREQ = (1)
7,ONE TO BE TAKEN TWICE A DAY,QUANT = (1) tobe FREQ = (2) FREQ = (1)
9,2 TABS AM 1 TABLET PM,QUANT = (2) am QUANT = (1)
14,TAKE one tds before meals,QUANT = (1) FREQ = (3)
16,2 IN THE MORNING 1 LUNCHTIME 1 TEATIME,QUANT = (2) inthemn QUANT = (1) lunchtime QUANT = (1)
19,2om,QUANT = (2) om
20,TAKE HALF TABLET ONCE DAILY,QUANT = (½) FREQ = (1) FREQ = (1)
21,one tid,QUANT = (1) FREQ = (3)


In [ ]:
       
# Work with the `medicationquantity` columns to see if you can determine the expected prescription duration. These rules assume that
# only one tablet / capsule is taken daily.
dplyr::mutate(
    quantity_transformed = 
        medicationquantity %>%
        
        # Transform X: Remove parentheses.
        stringr::str_replace("\\(?\\)?",                                                       "" ) %>%
        
        # Transform X: Make everything lower case.
        stringr::str_to_lower() %>%
        
        # Get rid of the drug mass, e.g. grams and mg.
        stringr::str_replace( "[0-9]*\\ ?(mg)|(gram)s?",              "" ) %>% # Why does "112 tablet - 4 mg + 1 gram" become "QUANT = (4 MONTHS) + 1"?
        
        
        # Transform X: Remove parentheses.
        stringr::str_replace( "(tab)(let)?s?",              "" ) %>%
        
        # Transform " pack of " to "*", which is used more often, it seems.
        stringr::str_replace( "\\ (pack)s?\\ (of)\\ ",              "*" ) %>%
        
        # Transform the implied multiplications.
        stringr::str_replace( "(^1\\*|x)|(\\*1\\ )",              " MULTI = (1 TIMES) " ) %>%
        stringr::str_replace( "(^2\\*|x)|(\\*2\\ )",              " MULTI = (2 TIMES) " ) %>%
        stringr::str_replace( "(^3\\*|x)|(\\*3\\ )",              " MULTI = (3 TIMES) " ) %>%
        stringr::str_replace( "(^4\\*|x)|(\\*4\\ )",              " MULTI = (4 TIMES) " ) %>%
        stringr::str_replace( "(^5\\*|x)|(\\*5\\ )",              " MULTI = (5 TIMES) " ) %>%
        stringr::str_replace( "(^6\\*|x)|(\\*6\\ )",              " MULTI = (6 TIMES) " ) %>%
        stringr::str_replace( "(^7\\*|x)|(\\*7\\ )",              " MULTI = (7 TIMES) " ) %>%
        stringr::str_replace( "(^8\\*|x)|(\\*8\\ )",              " MULTI = (8 TIMES) " ) %>%
        stringr::str_replace( "(^9\\*|x)|(\\*9\\ )",              " MULTI = (9 TIMES) " ) %>%
        stringr::str_replace( "(^10\\*|x)|(\\*10\\ )",              " MULTI = (10 TIMES) " ) %>%
        stringr::str_replace( "(^11\\*|x)|(\\*11\\ )",              " MULTI = (11 TIMES) " ) %>%
        stringr::str_replace( "(^12\\*|x)|(\\*12\\ )",              " MULTI = (12 TIMES) " ) %>%
        
        # Find realistic multiples of 28, which is a months' worth of tablets.
        stringr::str_replace( "(^|\\ )?(28)\\ ",              " QUANT = (1 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(56)\\ ",              " QUANT = (2 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(84)\\ ",              " QUANT = (3 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(112)\\ ",             " QUANT = (4 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(140)\\ ",             " QUANT = (5 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(168)\\ ",             " QUANT = (6 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(196)\\ ",             " QUANT = (7 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(224)\\ ",             " QUANT = (8 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(252)\\ ",             " QUANT = (9 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(280)\\ ",             " QUANT = (10 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(308)\\ ",             " QUANT = (11 MONTHS) " ) %>%
        stringr::str_replace( "(^|\\ )?(336)\\ ",             " QUANT = (12 MONTHS) " ) %>%
        
        # Tidy up the implied arithemtic operations.
        #stringr::str_replace( "\\ +\\ ",             "+" ) %>%
        stringr::str_replace( "(\\ \\*)|(\\*\\ )",             "*" ) %>%
        
        # Clean up scraps caused by previous transforms.
        stringr::str_replace( "\\ ?-\\ ?",             "" )
) %>%

## Extract my codes into columns for later calculation of dose.

The task is to extract codes like "FREQ = (1)" and "QUANT = (4)" in a meaningful, semantically-perserving way that permits me to calculate the dose.

For example:
- `2 twice daily` becomes `FREQ = (2) FREQ = (2) FREQ = (1) WHEN = (P.M.)`, which should return `MORNING(0) + EVENING(2 x 2 x 1)`, and eventually return `4`.
- `take one three times a day` becomes `QUANT = (1) QUANT = (3) FREQ = (1) WHEN = (P.M.)`, which should return `MORNING(0) + EVENING(1 x 3 x 1)`, and eventually return `3`.
- `one am and two pm` becomes `WHEN = (A.M.) + FREQ = (2) WHEN = (P.M.) WHEN = (P.M.)`, which should return `MORNING(1 x 1) + EVENING(2 x 1)`